Trying to start from scratch, I have deleted all concections to GitHub and now I have created this folder as the only thing conected to Github. This is the file main.ipynb all the other files will be .py and so will this eventually, but for now I am more comfortable with this as I can add all the notes I need.

In [ ]:
# This is the start of it all, these are the programsa used including the linked files. So this is the main file which will be run and then when needed 
# it will runn the other programs as needed.

import json
import hashlib
import random
import os
from user_profile import view_profile, update_contact_number
from fitness_data import add_fitness_data, view_fitness_data

# Initialize the user class. This will create a 5 digit number for each new member and save the name email phone number and encrypted password.
class User:
    existing_numbers = set()

    @staticmethod
    def generate_unique_number():
        while True:
            number = random.randint(10000, 99999)
            if number not in User.existing_numbers:
                User.existing_numbers.add(number)
                return number

    def __init__(self, first_name, second_name, email, contact_number, password, id=None):
        self.id = id if id is not None else User.generate_unique_number()
        self.first_name = first_name
        self.second_name = second_name
        self.email = email
        self.contact_number = contact_number
        self.password = password # Store the encrypted password, it is encrypted during the signing up section.
        self.data = {} # Additional data related to user. This can change, maybe notes, special requirments etc and we can add more

    def __repr__(self):
        return f"User(id={self.id}, first_name={self.first_name}, second_name={self.second_name}, email={self.email}, contact_number={self.contact_number})"

# Turnes all the info into a dictionary so we can store it in a Json file.
    def to_dict(self):
        return {
            'id': self.id,
            'first_name': self.first_name,
            'second_name': self.second_name,
            'email': self.email,
            'contact_number': self.contact_number,
            'password': self.password,
            'data': self.data
        }

# Turns all the data back from the Json file into usable data
    @classmethod
    def from_dict(cls, data):
        user = cls(
            id=data['id'],
            first_name=data['first_name'],
            second_name=data['second_name'],
            email=data['email'],
            contact_number=data['contact_number'],
            password=data['password']
        )
        user.data = data.get('data', {})
        return user

# Loads users from the Json file
    @staticmethod
    def load_users(filename):
        if not os.path.exists(filename):
            return []
        try:
            with open(filename, 'r') as file:
                content = file.read().strip()
                if content:
                    users_data = json.loads(content)
                    return [User.from_dict(data) for data in users_data]
        except (FileNotFoundError, json.JSONDecodeError):
            return []

# Saves the data to the json file
    @staticmethod
    def save_users(users, filename):
        with open(filename, 'w') as file:
            json.dump([user.to_dict() for user in users], file, indent=4)

# Find a user by email
    @staticmethod
    def find_user_by_email(users, email):
        for user in users:
            if user.email == email:
                return user
        return None

# Find a user by contact number
    @staticmethod
    def find_user_by_contact_number(users, contact_number):
        for user in users:
            if user.contact_number == contact_number:
                return user
        return None

# Add a new user
    @staticmethod
    def add_user(users, user):
        users.append(user)
        User.save_users(users, 'fitness_data.json')

# Signup function
def signup():
    email = input("Enter your email address: ")
    pwd = input("Enter your password: ")
    conf_pwd = input("Confirm your password: ")

    if conf_pwd == pwd:
        enc = conf_pwd.encode()
        hash1 = hashlib.md5(enc).hexdigest()

        if User.find_user_by_email(users, email):
            print("User already exists! with this email")
        else:
            first_name = input("Enter your first name: ")
            second_name = input("Enter your second name: ")
            contact_number = input("Enter your phone number: ")

            if User.find_user_by_contact_number(users, contact_number):
                print("User already exists with this phone number")
            else:
                new_user = User(first_name, second_name, email, contact_number, hash1)
                User.add_user(users, new_user)
                print("Welcome to your fitness tracker!")
    else:
        print("Your passwords do not match")

# Login function
def login():
    email = input("Enter your email: ")
    pwd = input("Enter your password: ")

    auth = pwd.encode()
    auth_hash = hashlib.md5(auth).hexdigest()

    user = User.find_user_by_email(users, email)

    if user and user.password == auth_hash:
        print("Welcome back!")
        user_menu(user)
    else:
        print("Invalid email or password. Please try again: ")

# User specific menu functions
def user_menu(user):
    while True:
        print(f"\nWelcome back {user.first_name}! What would you like to do?")
        print("'view' profile")
        print("'update' contact number")
        print("'add' fitness data")
        print("view fitness 'data'")
        print("'exit'")
        choice = input("Enter your choice: ")

        if choice == 'view':
            view_profile(user)
        elif choice == 'update':
            update_contact_number(user, users)
        elif choice == 'add':
            add_fitness_data(user, users)
        elif choice == 'data':
            view_fitness_data(user)
        elif choice == 'exit':
            print("Logging out ...")
            break
        else:
            print("Invalid choice. Please try again")

# Main code to run the login system.
users = User.load_users('fitness_data.json')

while True:
    print("\n******Login System*******")
    print("signup")
    print("login")
    print("exit")
    log = input("What would you like to do? ")

    if log == 'signup':
        signup()
    elif log == 'login':
        login()
    elif log == 'exit':
        print("Goodbye")
        break
    else:
        print("Invalid option, please try again")
